In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd

In [3]:
# selenium을 활용한 Instagram 게시글 접근 코드

# 로그인 정보
_INSTAGRAM_ID = 'snsa7007'
_INSTAGRAM_PW = 'zlddhkd123'

# 옵션 설정
options = webdriver.ChromeOptions()
dr = webdriver.Chrome(options=options)
dr.set_window_size(820, 1000)

# 인스타그램 로그인
dr.get('https://www.instagram.com/')
time.sleep(3)

input_id = dr.find_element(By.CSS_SELECTOR, 'input[name="username"]')
input_pw = dr.find_element(By.CSS_SELECTOR, 'input[name="password"]')
button_login = dr.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
input_id.send_keys(_INSTAGRAM_ID)
input_pw.send_keys(_INSTAGRAM_PW)
button_login.click()
time.sleep(5)

# 팝업 처리 (나중에 저장하기)
link_later_save = dr.find_element(By.CSS_SELECTOR, 'button._acan._acap._acas._aj1-._ap30')
link_later_save.click()
time.sleep(5)

# 맛집 검색 페이지로 이동 (키워드 변경해가면서.. ex: 여수맛집, 제주도맛집)
dr.get('https://www.instagram.com/explore/search/keyword/?q=%23여수맛집')
time.sleep(5)


In [4]:
# 첫번째 게시물 클릭하는 함수

def select_first(driver):
    first = driver.find_element(By.CSS_SELECTOR, "div._aagw")
    first.click()
    time.sleep(5)

In [5]:
# 다음 게시글로 넘어가는 함수

def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    right.click()
    time.sleep(3)

In [62]:
import random

# 좋아요 갯수가 숫자가 아닐 때 추정하는 함수

def estimate_likes_from_comments_and_tags(comments_count, tags_count):
    # 댓글과 태그 수를 기반으로 좋아요를 추정
    estimated_likes = (comments_count * 5) + (tags_count * 2) + random.randint(1, 10)
    return estimated_likes

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import re

# LLM을 이용하여 게시글로부터 식당 정보를 추출하는 코드
# GPT-3.5 turbo 모델 사용

api_key = "Your OpenAI API Key" # OpenAI API Key 입력
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-3.5-turbo", temperature=0.2)

# 프롬프트 템플릿 작성
prompt_template = """
다음 텍스트에서 식당 이름과 지역 주소를 각각 추출해 주세요.
출력 형식은 반드시 다음과 같아야 합니다:

- 식당 이름: [식당 이름1]
  지역 주소: [지역 주소1]
- 식당 이름: [식당 이름2]
  지역 주소: [지역 주소2]

* 주소 정보가 없는 경우 "없음"이라고 표시해 주세요.

텍스트: {content}
"""

# PromptTemplate을 사용해 템플릿 정의
template = PromptTemplate(input_variables=["content"], template=prompt_template)

# LLMChain 설정
llm_chain = LLMChain(prompt=template, llm=llm)

# 함수 정의: LLM으로부터 식당 이름과 주소를 추출
def extract_restaurant_info(content):
    # LLMChain을 통해 결과 생성
    result = llm_chain.predict(content=content)

    # 식당 이름이 여러 개 있는지 확인 (정확한 키워드 사용)
    restaurant_name = re.findall(r"식당 이름: (.*?)\n", result)

    # 식당 이름이 1개 초과인 경우 건너뛰기 (무분별한 추천 방지)
    if len(restaurant_name) > 1:
        return None  # 건너뛰기

    # 주소 추출
    address = re.findall(r"주소: (.*)", result)

    return [restaurant_name, address]

# 사용 예시 (함수 테스트)
content = "오늘 친구랑 서울 강남구 1033-11에 있는 '홍콩반점0410'에서 짜장면을 먹었어요. 강남역 근처라서 찾기 쉬웠어요!"
result = extract_restaurant_info(content)
print(result)

[['홍콩반점0410'], ['서울 강남구 1033-11']]


In [120]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import unicodedata
import re

def get_content(driver):
    # 본문 내용 가져오기
    try:
        # 본문 내용
        content_element = driver.find_element(By.CSS_SELECTOR, 'div._a9zs')
        content = content_element.text
        content = unicodedata.normalize('NFC', content)

        content = content.replace('\n', ' ')
        
        # 식당 정보 추출 (식당 이름과 주소)
        restaurant_info = extract_restaurant_info(content)
        if restaurant_info is None:
            return None
        else:
            restaurant_name = restaurant_info[0]
            restaurant_addr = restaurant_info[1]
            
    except Exception as e:
        content = ''
        restaurant_name = ''
        restaurant_addr = ''
        print(f"Error extracting content: {e}")

    
    # 본문 내용에서 해시태그 가져오기
    tags = re.findall(r'#[^\s#,\\]+', content)

    if not tags:  # tags 리스트가 비어 있으면 None
        tags = []

    try:
        # 댓글 가져오기
        comments_elements = driver.find_elements(By.CSS_SELECTOR, 'ul._a9ym li div._a9zs')
        comments = [unicodedata.normalize('NFC', c.text) for c in comments_elements]
        if not comments:
            comments = []
    except Exception as e:
        comments = []
        print(f"Error extracting comments: {e}")

    try:
        # 좋아요 수
        likes_element = driver.find_element(By.CSS_SELECTOR, 'span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.x1s688f.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj')
        likes = likes_element.text
        likes = unicodedata.normalize('NFC', likes)
        likes = re.findall(r'\d+', likes)[0]
        
    except Exception as e:
        likes = estimate_likes_from_comments_and_tags(len(comments), len(tags))
        print(f"error: {e}")

    date_tag = driver.find_element(By.TAG_NAME, 'time')
    date = date_tag.get_attribute('datetime')  # datetime 속성 가져오기

    # 수집한 정보 저장하기
    data = [content, likes, tags, comments, restaurant_name, restaurant_addr, date]
    
    return data


In [203]:
# 데이터 수집

select_first(dr) # 첫 게시글 열기
results = [] # 크롤링 결과를 저장할 리스트

In [132]:
def crawling_instagram(target):
    cnt = 0
    while cnt < target:
        try:
            data = get_content(dr)
            if data is not None:
                results.append(data)
                cnt += 1
                print(cnt)
            move_next(dr)

        except:
            time.sleep(7)
            move_next(dr)

In [204]:
crawling_instagram(300)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# 결과 리스트를 데이터프레임으로 변환

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'likes','hashtags', 'comments', 'restaurant name', 'address', 'date']
results_df.drop_duplicates(subset=['content'], inplace=True) # 광고 등 중복 게시글 제거

In [8]:
import emoji

# 이모지 등 특수문자 제거 함수
def remove_emoji(text):
    text = emoji.replace_emoji(text, replace='')
    return text

In [206]:
# 이모지 제거 및 빈칸 삭제
# 데이가 깨지지 않고 저장될 수 있도록 함

results_df['content'] = results_df['content'].apply(remove_emoji)
results_df['hashtags'] = results_df['hashtags'].apply(lambda tags: [remove_emoji(tag) for tag in tags] if tags else [])
results_df['comments'] = results_df['comments'].apply(lambda comments: [remove_emoji(comment) for comment in comments])
results_df['comments'] = results_df['comments'].apply(lambda x: '' if x == [] else x)
results_df['restaurant name'] = results_df['restaurant name'].apply(lambda restaurant_name: [remove_emoji(name) for name in restaurant_name])
results_df['address'] = results_df['address'].apply(lambda addresses: [remove_emoji(address) for address in addresses])
results_df

,content,likes,hashtags,comments,restaurant name,address,date
0,부산 커플들 무조건 가야하는 파스타 전문점 #플랜온서면점 오픈한지 얼마안돼서 매장...,297,"[#플랜온서면점, #부산맛집, #서면맛집, #전포맛집, #부산가볼만한곳, #부산여행...",[],[플랜온서면점],[부산 부산진구 중앙대로680번가길 38 2층],2024-11-26T05:58:15.000Z
1,댓글달면 4만원 식사권 증정 김해에서 이미 핫플로 유명한 양식 맛집 부산에 상륙 ...,140,"[#플랜온서면점, #부산맛집, #서면맛집, #전포맛집, #부산가볼만한곳, #부산여행...","[너무 맛있겠자나요, 정겹고 고급진 브런치 타임 즐기고 싶어요~, 헐 대박 가보고 ...",[플랜온서면점],[부산 부산진구 중앙대로680번가길 38 2층],2024-11-29T01:26:21.000Z
2,눈과 입이 즐거운 분위기 이쁜 카페 #무화 아늑하고 조용한 분위기에서 잔잔하게 힐...,159,"[#무화, #부산맛집, #부산카페, #부산가볼만한곳, #부산여행]",,[무화],[부산 부산진구 서전로67번길 27-9 1층],2024-11-28T08:00:15.000Z
3,당장 가야하는 존예 겨울 축제 #광복로겨울빛트리축제 트리도 보고 화려한 미디어 아트...,150,"[#광복로겨울빛트리축제, #부산축제, #부산겨울축제, #부산가볼만한곳, #부산이색축...",,[없음],[없음],2024-11-26T08:00:15.000Z
4,"쫀득쫀득 수제 마카롱 맛집 #마카러버 마카롱 러버들 환장하는 쫀득 마카롱 찐맛집,...",151,"[#마카러버, #부산맛집, #부산카페, #부산가볼만한곳, #부산여행]",[비쥴때매 못 먹.. 넘 긔욤뽀짣],[마카러버],"[부산 금정구 부곡온천천로 170 상가 102호, 마카러버]",2024-11-26T08:00:13.000Z
...,...,...,...,...,...,...,...
295,이미 입소문 자자한 브런치 카페 #듀플릿해리단길 이제 크리스마스 한달 남은 거알지...,201,"[#듀플릿해리단길, #부산맛집, #부산카페, #부산가볼만한곳, #부산여행]","[@_june_20 헉, @lovesome_zisoo]",[브런치 카페 #듀플릿해리단길],[부산 해운대구 우동1로20번길 27-13 1층],2023-11-26T08:00:25.000Z
296,바다를 보며 먹는 전복 가정식 #어느 멋진날 여기는 바닷가 바로 앞이라서 바다의 ...,281,"[#어느, #부산맛집, #부산카페, #부산가볼만한곳, #부산여행]","[@6uuuuuuk 브레이크타임 잇는듯!!!, @_june_20 냐밍]",[바다를 보며 먹는 전복 가정식],[부산 기장군 일광읍 기장해안로 1286 일광 투썸플레이스 옆 옆집],2023-11-22T08:00:28.000Z
297,바다를 품은 대형카페 #카페385 영도 뷰 맛집으로 완전 강추 ! 베이커리 종류도...,178,"[#카페385, #부산맛집, #부산카페, #부산가볼만한곳, #부산여행]",[여긴 밤 루프탑이 찐맛집임..],[카페385],[부산 영도구 태종로 539],2023-11-20T08:00:34.000Z
298,양산에서 중식의 진가를 보여준 찐맛집 #만리장성중국관 간짜장/ 짬뽕 맛집으로 양산...,291,"[#만리장성중국관, #양산맛집, #범어맛집, #양산가볼만한곳, #양산짬뽕, #범어짬...",,[만리장성중국관],[양산시 물금읍 화합2길3-1],2023-11-20T07:29:20.000Z
